In [1]:
import requests
import json
import time
from datetime import date
import Utils as utils
import pandas as pd
import _thread

## Run measurements

In [2]:
df = utils.getAvailablePCProbesCC()

df = df.loc[df['CountryCode'].isin(utils.ccs)]
    
available_ccs = (df.groupby(['CountryCode']).size().to_frame(name = 'count').reset_index()).CountryCode.values

df.loc[df['CountryCode']=='ZA'].head()


,ASN,CityName,ConnectionType,CountryCode,DNSResolver,GeolocationAccuracy,IPAddress,Latitude,Longitude,Network,NetworkID,Platform,ProbeID,Screensize,Version
26,37457,Pretoria,Ethernet,ZA,196.43.42.231,20000,105.229.3.153,-25.76,28.14,Telkom Internet,422,PC,cea2de8f-4491-4e6a-84e9-578618b6a303,1280x1024,1.0.44.0
41,37457,Durban,Wireless80211,ZA,196.43.54.230,20000,105.228.56.152,-29.86,31.04,Telkom Internet,422,PC,9096cfe7-0148-4fe3-a244-b84ca7a36e7d,1366x768,1.0.44.0
48,37179,Cape Town,Ethernet,ZA,74.125.47.8,25000,41.66.138.51,-33.90,18.63,Africainx,104056,PC,cdd6cac8-9b14-482e-bcfe-42327f17eb73,1920x1080,1.0.45.0
61,37611,Cape Town,Wireless80211,ZA,169.1.2.11,10000,165.73.108.28,-33.92,18.42,Afrihost,1759,PC,6b2aaf0c-9f24-4344-bfde-13a4ad94cc74,1366x768,1.0.45.0
66,37457,Johannesburg,Ethernet,ZA,196.43.42.231,10000,105.186.113.166,-26.23,28.13,Telkom Internet,422,PC,627b3e16-ba7d-4768-a121-28017acbf3a8,1920x1080,1.0.45.0


In [3]:
df.loc[df['CountryCode']=='ZA'].head()

,ASN,CityName,ConnectionType,CountryCode,DNSResolver,GeolocationAccuracy,IPAddress,Latitude,Longitude,Network,NetworkID,Platform,ProbeID,Screensize,Version
26,37457,Pretoria,Ethernet,ZA,196.43.42.231,20000,105.229.3.153,-25.76,28.14,Telkom Internet,422,PC,cea2de8f-4491-4e6a-84e9-578618b6a303,1280x1024,1.0.44.0
41,37457,Durban,Wireless80211,ZA,196.43.54.230,20000,105.228.56.152,-29.86,31.04,Telkom Internet,422,PC,9096cfe7-0148-4fe3-a244-b84ca7a36e7d,1366x768,1.0.44.0
48,37179,Cape Town,Ethernet,ZA,74.125.47.8,25000,41.66.138.51,-33.90,18.63,Africainx,104056,PC,cdd6cac8-9b14-482e-bcfe-42327f17eb73,1920x1080,1.0.45.0
61,37611,Cape Town,Wireless80211,ZA,169.1.2.11,10000,165.73.108.28,-33.92,18.42,Afrihost,1759,PC,6b2aaf0c-9f24-4344-bfde-13a4ad94cc74,1366x768,1.0.45.0
66,37457,Johannesburg,Ethernet,ZA,196.43.42.231,10000,105.186.113.166,-26.23,28.13,Telkom Internet,422,PC,627b3e16-ba7d-4768-a121-28017acbf3a8,1920x1080,1.0.45.0


In [7]:
utils.getProbesByCC(df,'ZA')

[{'ProbeID': '0263e472-fe36-44ec-9d67-e24731a88204',
  'Latitude': -26.23,
  'Longitude': 28.06,
  'Screensize': '1366x768'},
 {'ProbeID': '2ec75c21-6154-446c-a898-e1d5b770beb9',
  'Latitude': -25.76,
  'Longitude': 28.14,
  'Screensize': nan},
 {'ProbeID': '37e7858a-0ff8-4fdc-8d15-14c6e4ef92ce',
  'Latitude': -29.86,
  'Longitude': 31.04,
  'Screensize': nan},
 {'ProbeID': '5667f00d-a444-4607-8f5e-fcdb9a8b1114',
  'Latitude': -33.03,
  'Longitude': 27.92,
  'Screensize': '1920x1080'},
 {'ProbeID': '588a8bd3-e68d-4821-9618-2aade834dc7a',
  'Latitude': -33.97,
  'Longitude': 25.58,
  'Screensize': '1366x768'},
 {'ProbeID': 'a209efd4-7d25-4da5-b0ae-c206188c91ff',
  'Latitude': -25.76,
  'Longitude': 28.14,
  'Screensize': '1280x1024'},
 {'ProbeID': '5b482a4c-a0df-4c9c-8214-0980a24bc0f3',
  'Latitude': -29.85,
  'Longitude': 31.02,
  'Screensize': '1360x768,1366x768'},
 {'ProbeID': '627b3e16-ba7d-4768-a121-28017acbf3a8',
  'Latitude': -26.1,
  'Longitude': 28.25,
  'Screensize': '1920x108

In [9]:
for cc in available_ccs:

    probes = df.loc[df['CountryCode']==cc]

    if (probes is None):
        continue
    
    df_testID = pd.DataFrame(columns=['videoID', 'cc', 'testID', 'probeID', 'Latitude', 'Longitude', 'Resolution'])
    
    for index, p in probes.iterrows():
        videos = utils.getVideoList(str(p['Latitude']), str(p['Longitude']), '10km', '5')
        #print(videos)

        for v in videos:

            for res in ['SMALL_RES', 'MEDIUM_RES', 'LARGE_RES']:

                testID = utils.runVideoTest(v, p['ProbeID'], res)
                
                if (testID is None):
                    break

                df_testID = df_testID.append({'videoID': v, 'cc': cc, 'testID': testID, 
                                              'probeID': p['ProbeID'], 'Latitude': p['Latitude'], 
                                              'Longitude' : p['Longitude'], 'Resolution' : res}, ignore_index=True)
                
               

                print(v, cc, testID, p['ProbeID'], p['Latitude'], p['Longitude'], res)
        
df_testID.to_csv('data/testID-11062019.csv', sep='|', encoding='utf-8', index=False)   



http://pcsucdn.com/YoutubeVideoTest2.html?v=dXJoFvDGvGA&pw=320&ph=240&pq=small
dXJoFvDGvGA AO 0a7f5ef7-c66e-4ddc-923d-157f37e8ad3b 3387de05-2a8d-47cb-b001-4204a5371efc -8.84 13.23 SMALL_RES
http://pcsucdn.com/YoutubeVideoTest2.html?v=dXJoFvDGvGA&pw=640&ph=360&pq=medium
dXJoFvDGvGA AO c25bc705-df4a-4fb6-82ec-016e6e865cc5 3387de05-2a8d-47cb-b001-4204a5371efc -8.84 13.23 MEDIUM_RES
http://pcsucdn.com/YoutubeVideoTest2.html?v=dXJoFvDGvGA&pw=853&ph=480&pq=large
dXJoFvDGvGA AO 75aec453-ece6-43a5-b440-dfb83396e6bd 3387de05-2a8d-47cb-b001-4204a5371efc -8.84 13.23 LARGE_RES
http://pcsucdn.com/YoutubeVideoTest2.html?v=F7TCqMiqQxg&pw=320&ph=240&pq=small
F7TCqMiqQxg AO bb70bd17-1ece-48b9-b7b5-f916deb1223c 3387de05-2a8d-47cb-b001-4204a5371efc -8.84 13.23 SMALL_RES
http://pcsucdn.com/YoutubeVideoTest2.html?v=F7TCqMiqQxg&pw=640&ph=360&pq=medium
F7TCqMiqQxg AO f467ad9b-e582-44fe-8696-61d3c5ad1688 3387de05-2a8d-47cb-b001-4204a5371efc -8.84 13.23 MEDIUM_RES
http://pcsucdn.com/YoutubeVideoTest2.html?v=F7

## Collect results

In [2]:
print(utils.getAvailablePCProbesCC())

['BI' 'BW' 'DJ' 'DZ' 'EG' 'ET' 'KE' 'KM' 'LY' 'MA' 'NA' 'NG' 'SN' 'SO'
 'TN' 'ZA' 'ZW']


In [2]:
print(utils.getProbes('MU'))

[{'ProbeID': 'fb94eede-9aff-43fa-96ee-1f944129fcda', 'Latitude': -20.28, 'Longitude': 57.55}, {'ProbeID': '14a8ea0a-ff4a-4538-9880-e6fbfb12d8da', 'Latitude': -20.11, 'Longitude': 57.58}, {'ProbeID': 'af99e32f-37d4-466c-a807-bb93f91f565c', 'Latitude': -20.24, 'Longitude': 57.48}]


In [2]:
print(utils.runVideoTest('dkBoUC0RAT0', '14a8ea0a-ff4a-4538-9880-e6fbfb12d8da'))

589ddfac-2322-438c-83f9-8405f25713e7


In [2]:
print(utils.getProbes('MU'))

None


In [ ]:
retrieveVideoTestResults('5ead3b75-e21f-4390-a4bd-0e9799aa05d0')

In [5]:
print(utils.runVideoTest('8aYpKlhXDvE', 'b7f37d52-c167-4469-becb-725be1034fa5', 'NOT_SPECIFIED_RES'))

http://pcsucdn.com/YoutubeVideoTest2.html?v=8aYpKlhXDvE

